In [ ]:
# AutoGluon Text 분석
# https://auto.gluon.ai/stable/tutorials/multimodal/text_prediction/beginner_text.html#sentiment-analysis-task

# Hyperparmeter 최적화
# https://auto.gluon.ai/stable/tutorials/multimodal/advanced_topics/customization.html

In [1]:
!pip3 install -U pip
!pip3 install -U setuptools wheel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip3 install torch==1.13.1+cpu torchvision==0.14.1+cpu -f https://download.pytorch.org/whl/cpu/torch_stable.html
!pip3 install torch==1.13.1+cu116 torchvision==0.14.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cpu/torch_stable.html
  Using cached https://download.pytorch.org/whl/cpu/torch-1.13.1%2Bcpu-cp38-cp38-linux_x86_64.whl (199.1 MB)
  Using cached https://download.pytorch.org/whl/cpu/torchvision-0.14.1%2Bcpu-cp38-cp38-linux_x86_64.whl (16.8 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.14.1+cu116
    Uninstalling torchvision-0.14.1+cu116:
      Successfully uninstalled torchvision-0.14.1+cu116
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu116
  Using cached https://download.pytorch.org/whl/cu116/torch-1.13.1%2Bcu116-cp38-cp38-linux_x86_64.whl (1

In [3]:
!pip3 install autogluon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

PATH = "/content/drive/MyDrive/ML-Olympiad/Detect-ChatGpt-answers/"

train = pd.read_csv(PATH+'data/train.csv')
test = pd.read_csv(PATH+'data/test.csv')
submission = pd.read_csv(PATH+'data/sample_submission.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
train

,ID,prompt,answer,AI
0,690.0,What is the future of AI?,The future of Artificial Intelligence (AI) is ...,1.0
1,304.0,What is your biggest challenge in your career?,My biggest challenge in my career is balancing...,0.0
2,63.0,What is the tallest mountain in the world?,The tallest mountain in the world is Mount Eve...,1.0
3,894.0,What are the best 5 super cars?,McLaren 720S.\r\nFerrari 296 GTB.\r\nAudi R8.\...,0.0
4,796.0,What is deep learning?,a type of machine learning based on artificial...,0.0
...,...,...,...,...
743,283.0,What is the most important thing in a family?,The most important thing in a family for me is...,0.0
744,895.0,What are the best machine learning Models?,Linear regression.\r\nLogistic regression\r\nD...,0.0
745,963.0,How can I chop onions without crying?,The best thing to do is very simple: use a ver...,0.0
746,156.0,What is the best way to stay healthy?,The best way to stay healthy varies from perso...,0.0


## 1. 전처리

In [6]:
train["prompt"].isna().sum()

0

In [7]:
# train = train[train["prompt"].isna()==False]

In [8]:
train["answer"].isna().sum()

1

In [9]:
train = train[train["answer"].isna()==False]

In [10]:
train["AI"].isna().sum()

1

In [11]:
train = train[train["AI"].isna()==False]

In [12]:
test.isna().sum()

ID        0
prompt    0
answer    0
dtype: int64

In [13]:
train

,ID,prompt,answer,AI
0,690.0,What is the future of AI?,The future of Artificial Intelligence (AI) is ...,1.0
1,304.0,What is your biggest challenge in your career?,My biggest challenge in my career is balancing...,0.0
2,63.0,What is the tallest mountain in the world?,The tallest mountain in the world is Mount Eve...,1.0
3,894.0,What are the best 5 super cars?,McLaren 720S.\r\nFerrari 296 GTB.\r\nAudi R8.\...,0.0
4,796.0,What is deep learning?,a type of machine learning based on artificial...,0.0
...,...,...,...,...
743,283.0,What is the most important thing in a family?,The most important thing in a family for me is...,0.0
744,895.0,What are the best machine learning Models?,Linear regression.\r\nLogistic regression\r\nD...,0.0
745,963.0,How can I chop onions without crying?,The best thing to do is very simple: use a ver...,0.0
746,156.0,What is the best way to stay healthy?,The best way to stay healthy varies from perso...,0.0


In [14]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
import numpy as np

In [17]:
import re
import emoji
from unidecode import unidecode

#remove html tags
def removeHTML(x):
    html=re.compile(r'<.*?>')
    return html.sub(r'',x)
    
#remove emoji and symbols
def dataPreprocessing(x): 
    x = x.lower()
    x = removeHTML(x)
    x = emoji.demojize(x, delimiters=(" ", " "))
    x = re.sub("'re", ' are',x) # 're n't --> are not 
    x = re.sub("n't", ' not',x) # 're n't --> are not 
    x = re.sub(r'[^A-Za-z0-9!.?, ]', '', x) 
    x = re.sub(r'[!.?]','/n',x)
    x = re.sub(r'[,]',' ',x)

    # x = re.sub("@\w+", '',x) # removing mentions (@)
    # x = re.sub("'\d+", '',x)
    # x = re.sub("\d+", '',x)
    # x = re.sub(r"[^\w\s]", '',x) # to remove symbols
    # x = re.sub("http\w+", '',x)
    # x = re.sub("\s[a-z]\s", '',x)
    x = re.sub("[\n|\r]",' ',x) # index 3번 \r, \n 삭제
    x = re.sub("[ ]+",' ',x) # space*n --> space
    x = re.sub("_",'',x) # _people_ --> people
    x = x.strip()
    return unidecode(x) #emily bronté --> emily bronte, mi² --> mi2, πr --> pr

In [18]:
train_text = train.copy()
train_text["prompt"]=train["prompt"].apply(lambda x: dataPreprocessing(x))
train_text["answer"]=train["answer"].apply(lambda x: dataPreprocessing(x))

In [19]:
test_text = test.copy()
test_text["prompt"]=test["prompt"].apply(lambda x: dataPreprocessing(x))
test_text["answer"]=test["answer"].apply(lambda x: dataPreprocessing(x))

In [20]:
train_text

,ID,prompt,answer,AI
0,690.0,what is the future of ai/n,the future of artificial intelligence ai is bo...,1.0
1,304.0,what is your biggest challenge in your career/n,my biggest challenge in my career is balancing...,0.0
2,63.0,what is the tallest mountain in the world/n,the tallest mountain in the world is mount eve...,1.0
3,894.0,what are the best 5 super cars/n,mclaren 720s/nferrari 296 gtb/naudi r8/nmasera...,0.0
4,796.0,what is deep learning/n,a type of machine learning based on artificial...,0.0
...,...,...,...,...
743,283.0,what is the most important thing in a family/n,the most important thing in a family for me is...,0.0
744,895.0,what are the best machine learning models/n,linear regression/nlogistic regressiondecision...,0.0
745,963.0,how can i chop onions without crying/n,the best thing to do is very simple use a very...,0.0
746,156.0,what is the best way to stay healthy/n,the best way to stay healthy varies from perso...,0.0


In [21]:
from sklearn.model_selection import train_test_split

X = train_text.drop(['ID'], axis=1)
y = train['AI']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
pd.DataFrame(X_train, columns=['prompt', 'answer'])

,prompt,answer
70,what is the name of the unit used to measure p...,the unit used to measure power is the watt/n
164,what is the name of the largest underwater cav...,sac actun cave system
712,what is the name of the gas that makes up the ...,the gas that makes up the majority of the eart...
265,what is the name of the largest underwater cav...,after doing a lot of research i found that sac...
250,what is the most important factor in making a ...,the most important factor in making a decision...
...,...,...
71,what is your biggest strength/n,my biggest strength is my ability to communica...
106,what is the currency of australia/n,australian dollar/n
270,what is the best way to stay positive/n,the best way to stay positive in my opinion is...
435,what is the name of the largest underwater cav...,sac actun cave system


In [23]:
pd.DataFrame(y_train, columns=['AI'])

,AI
70,1.0
164,0.0
712,0.0
265,0.0
250,0.0
...,...
71,0.0
106,0.0
270,0.0
435,0.0


In [24]:
train_data = pd.concat([pd.DataFrame(X_train, columns=['prompt', 'answer']), pd.DataFrame(y_train, columns=['AI'])], axis=1)
train_data

,prompt,answer,AI
70,what is the name of the unit used to measure p...,the unit used to measure power is the watt/n,1.0
164,what is the name of the largest underwater cav...,sac actun cave system,0.0
712,what is the name of the gas that makes up the ...,the gas that makes up the majority of the eart...,0.0
265,what is the name of the largest underwater cav...,after doing a lot of research i found that sac...,0.0
250,what is the most important factor in making a ...,the most important factor in making a decision...,0.0
...,...,...,...
71,what is your biggest strength/n,my biggest strength is my ability to communica...,0.0
106,what is the currency of australia/n,australian dollar/n,0.0
270,what is the best way to stay positive/n,the best way to stay positive in my opinion is...,0.0
435,what is the name of the largest underwater cav...,sac actun cave system,0.0


In [25]:
test_data = pd.concat([pd.DataFrame(X_test, columns=['prompt', 'answer']), pd.DataFrame(y_test, columns=['AI'])], axis=1)
test_data

,prompt,answer,AI
208,what is the most important lesson you have lea...,the most important lesson i have learned in li...,0.0
259,what is your biggest challenge in life/n,my biggest challenge in life is balancing my w...,0.0
97,where to put code documentation/n,some use other systems take a look at pythons ...,0.0
148,what is the key to success in the culinary arts/n,the key to success in the culinary arts is cre...,0.0
395,what is your take on the current state of the ...,the current state of the world is complex and ...,1.0
...,...,...,...
554,what is the best way to stay organized/n,the best way to stay organized varies from per...,0.0
69,what is the name of the largest living structu...,the largest living structure in the world is t...,1.0
522,what is the meaning of life/n,the meaning of life is subjective and can vary...,0.0
131,what is natural language processing nlp/n,nlp is an interdisciplinary subfield of lingui...,0.0


## 2. 모델링

In [26]:
label = 'AI'
eval_metric = 'acc'
time_limit = 3600 * 2

In [27]:
from ray import tune

In [28]:
hyperparameters = {
        # "optimization.learning_rate": tune.uniform(0.00005, 0.005),
        # "optimization.optim_type": tune.choice(["adamw", "sgd"]),
        # "optimization.max_epochs": tune.choice(["40", "50"]),
        # "model.timm_image.checkpoint_name": tune.choice(["swin_base_patch4_window7_224", "convnext_base_in22ft1k"])
}
hyperparameter_tune_kwargs = {
    # "searcher": "bayes", # random
    # "scheduler": "ASHA",
    # "num_trials": 2,
}

In [29]:
from autogluon.multimodal import MultiModalPredictor
# 모듈 못찾는 Attribute Error 발생시 런타임 초기화 후 코드 처음부터 재실행
# EX. AttributeError: module 'PIL.Image' has no attribute 'Resampling'
import uuid

model_path = PATH+f"model/{uuid.uuid4().hex}-automm_sst_QA_unicode_2hours"
predictor = MultiModalPredictor(label=label, eval_metric=eval_metric, path=model_path)
# predictor = MultiModalPredictor(label=label, eval_metric=eval_metric)
predictor.fit(train_data, presets='best_quality', 
              hyperparameters={"optimization.max_epochs": 1000, "optimization.patience": 500},
              time_limit=time_limit)

INFO:lightning_fabric.utilities.seed:Global seed set to 123
AutoMM starts to create your model. ✨

- Model will be saved to "/content/drive/MyDrive/ML-Olympiad/Detect-ChatGpt-answers/model/49c1c29e58334c01b934df32dc6aea89-automm_sst_QA_unicode_2hours".

- Validation metric is "acc".

- To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir /content/drive/MyDrive/ML-Olympiad/Detect-ChatGpt-answers/model/49c1c29e58334c01b934df32dc6aea89-automm_sst_QA_unicode_2hours
    ```

Enjoy your coffee, and let AutoMM do the job ☕☕☕ Learn more at https://auto.gluon.ai



/usr/local/lib/python3.8/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit None Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 83, global step 333: 'val_acc' was not in top 3
Start to fuse 3 checkpoints via the greedy soup algorithm.


Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

AutoMM has created your model 🎉🎉🎉

- To load the model, use the code below:
    ```python
    from autogluon.multimodal import MultiModalPredictor
    predictor = MultiModalPredictor.load("/content/drive/MyDrive/ML-Olympiad/Detect-ChatGpt-answers/model/49c1c29e58334c01b934df32dc6aea89-automm_sst_QA_unicode_2hours")
    ```

- You can open a terminal and launch Tensorboard to visualize the training log:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir /content/drive/MyDrive/ML-Olympiad/Detect-ChatGpt-answers/model/49c1c29e58334c01b934df32dc6aea89-automm_sst_QA_unicode_2hours
    ```

- If you are not satisfied with the model, try to increase the training time, 
adjust the hyperparameters (https://auto.gluon.ai/stable/tutorials/multimodal/advanced_topics/customization.html),
or post issues on GitHub: https://github.com/autogluon/autogluon




## 3. 성능평가

In [ ]:
# 모델 성능평가
# predictor.leaderboard(silent = True)

In [30]:
test_score = predictor.evaluate(test_data)
print(test_score)

Predicting: 0it [00:00, ?it/s]

{'acc': 0.8466666666666667}


In [31]:
# model_to_use = predictor.get_model_best()
model_pred = predictor.predict(test_text)

Predicting: 0it [00:00, ?it/s]

In [32]:
submission['Category'] = model_pred

In [33]:
submission = submission.astype({'Category':'int64'})

In [34]:
submission

,Id,Category
0,710,1
1,487,0
2,136,1
3,44,1
4,627,0
...,...,...
244,702,1
245,500,0
246,818,0
247,584,0


In [35]:
submission.to_csv(PATH+'submit/autoGluon_QA_unicode_2hours_submission_230305.csv', index=False)

In [36]:
model = predictor.dump_model()

Model weights for hf_text are saved to /content/drive/MyDrive/ML-Olympiad/Detect-ChatGpt-answers/model/49c1c29e58334c01b934df32dc6aea89-automm_sst_QA_unicode_2hours/hf_text.
Tokenizer hf_text saved to /content/drive/MyDrive/ML-Olympiad/Detect-ChatGpt-answers/model/49c1c29e58334c01b934df32dc6aea89-automm_sst_QA_unicode_2hours/hf_text.


In [37]:
import pickle

# 모델 객체 저장
with open(PATH+'model/autoGluon_QA_unicode_2hours_model_230305.pkl', 'wb') as f:
    pickle.dump(model, f)

In [38]:
from joblib import dump, load

# 모델 객체 저장
dump(model, PATH+'model/autoGluon_QA_unicode_2hours_model_230305.joblib')

# 저장된 모델 객체 로드
# model = load('model.joblib')

['/content/drive/MyDrive/ML-Olympiad/Detect-ChatGpt-answers/model/autoGluon_QA_unicode_2hours_model_230305.joblib']